In [1]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power

prompt = "A cat holding a sign that says hello world"
image = pipe(
    prompt,
    height=1024,
    width=1024,
    guidance_scale=3.5,
    num_inference_steps=50,
    max_sequence_length=512,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]
image.save("flux-dev.png")


GatedRepoError: 401 Client Error. (Request ID: Root=1-66baa01e-5529396114888a7f77220b97;e2a2dc67-d6e5-4641-8c9e-b93cbd6aa6d8)

Cannot access gated repo for url https://huggingface.co/black-forest-labs/FLUX.1-dev/resolve/main/model_index.json.
Access to model black-forest-labs/FLUX.1-dev is restricted. You must be authenticated to access it.